<a href="https://colab.research.google.com/github/kangma07/browser-2020/blob/master/%EC%82%B0%ED%95%99.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import math
import numpy as np

# get center of contour
def centroid(contour):
    M = cv2.moments(contour);
    cx = int(round(M['m10']/M['m00']));
    cy = int(round(M['m01']/M['m00']));
    center = (cx, cy);
    return center;

# load image
img = cv2.imread("target.png");
img = img[:,:-1,:]; # there's a bit of wall or something on the right

# hsv
hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV);
h,s,v = cv2.split(hsv);

# find the outer ring
v_mask = cv2.inRange(v, 0, 155);

# contours
_, contours, _ = cv2.findContours(v_mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE);

# find the biggest contour
biggest_cntr = None;
biggest_area = 0;
marked = img.copy();x``
for contour in contours:
    area = cv2.contourArea(contour);
    if area > biggest_area:
        biggest_area = area;
        biggest_cntr = contour;
cv2.drawContours(marked, [biggest_cntr], -1, (0, 255, 0), 3);

# find biggest radius
big_radius = math.sqrt(biggest_area / math.pi);

# find center
center_v_mask = cv2.inRange(v, 215, 255);
_, contours, _ = cv2.findContours(center_v_mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE);
center = centroid(contours[0]);

# use this to calculate the middle radius
area = cv2.contourArea(contours[0]);
little_radius = math.sqrt(area / math.pi);

# draw center
marked = cv2.circle(marked, center, 2, (155,155,0), -1);

# mask holes
h_mask = cv2.inRange(h, 0, 30);
h_mask = cv2.medianBlur(h_mask, 11);

# draw contour centers
_, contours, _ = cv2.findContours(h_mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE);
holes = [];
for contour in contours:
    c = centroid(contour);
    holes.append(c);
    marked = cv2.circle(marked, c, 2, (0,0,155), -1);

# calculate approximate size of each ring
# (with foreknowledge that there are 9 + bullseye)
remaining_radius = big_radius - little_radius;
slices = remaining_radius / 9;

# calculate scores
scores = [];
for hole in holes:
    # get distance from center
    dx = hole[0] - center[0];
    dy = hole[1] - center[1];
    dist = math.sqrt(dx*dx + dy*dy);

    # check score
    dist -= little_radius;
    if dist < 0:
        scores.append(10);
    else:
        scores.append(9 - int(dist / slices));

# draw the scores
font = cv2.FONT_HERSHEY_SIMPLEX ;
for a in range(len(holes)):
    tup = (holes[a][0], holes[a][1]);
    marked = cv2.putText(marked, str(scores[a]), tup, font, 1, (0,0,155), 2, cv2.LINE_AA);

# show
cv2.imshow("marked", marked);
cv2.waitKey(0);